In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import random
from collections import deque

In [2]:
train = pd.read_csv("train.csv")
sim =  pd.read_csv("sim.csv")
first_data = pd.read_csv('first_minus_data_for_ml.csv', index_col=0)
first_data['D_mean'] = first_data.apply(lambda x: x.D_sum / x.D_count, axis=1)
first_data['K_mean'] = first_data.apply(lambda x: x.K_sum / x.K_count, axis=1)
first_data

class       D_sum      D_count    D_mean       K_sum      K_count  \
id                                                                         
0        NaN -405.776287  2756.500000 -0.147207 -367.086575  1339.000000   
1        NaN -665.306470  2913.833333 -0.228327 -164.958415  1236.166667   
2        NaN -349.802807  2779.833333 -0.125836 -226.382758  1319.666667   
3        NaN -451.004127  2890.333333 -0.156039 -135.502683  1256.000000   
4        NaN -445.659327  2794.000000 -0.159506 -381.058482  1352.333333   
...      ...         ...          ...       ...         ...          ...   
14995    NaN -436.845293  2759.333333 -0.158316 -412.016401  1344.333333   
14996    NaN -552.881760  2931.500000 -0.188600 -137.054831  1231.166667   
14997    NaN -487.924329  2918.500000 -0.167183 -171.393176  1261.166667   
14998    NaN -455.885051  2907.833333 -0.156778 -214.564370  1316.666667   
14999    NaN -907.451780  2908.833333 -0.311964 -206.534707  1193.666667   

         K_mean  
id               
0     -0.274150  
1     -0.133444  
2     -0.171545  
3     -0.107884  
4     -0.281779  
...         ...  
14995 -0.306484  
14996 -0.111321  
14997 -0.135900  
14998 -0.162960  
14999 -0.173025  

[15000 rows x 7 columns]

In [3]:
pred_classes = pd.DataFrame({'id': np.arange(0, 15000),
                'class': [np.nan] * 15000,
                'D_sum': np.zeros(15000),
                'D_count': np.zeros(15000),
                'D_mean': np.zeros(15000),
                'D_squarred_difference': np.zeros(15000),
                'D_standard_error': np.zeros(15000),
                'D_max': -np.ones(15000),
                'D_min': np.zeros(15000),
                'K_sum': np.zeros(15000),
                'K_count': np.zeros(15000),
                'K_mean': np.zeros(15000),
                'K_squarred_difference': np.zeros(15000),
                'K_standard_error': np.zeros(15000),
                'K_max': -np.ones(15000),
                'K_min': np.zeros(15000)})
pred_classes

id  class  D_sum  D_count  D_mean  D_squarred_difference  \
0          0    NaN    0.0      0.0     0.0                    0.0   
1          1    NaN    0.0      0.0     0.0                    0.0   
2          2    NaN    0.0      0.0     0.0                    0.0   
3          3    NaN    0.0      0.0     0.0                    0.0   
4          4    NaN    0.0      0.0     0.0                    0.0   
...      ...    ...    ...      ...     ...                    ...   
14995  14995    NaN    0.0      0.0     0.0                    0.0   
14996  14996    NaN    0.0      0.0     0.0                    0.0   
14997  14997    NaN    0.0      0.0     0.0                    0.0   
14998  14998    NaN    0.0      0.0     0.0                    0.0   
14999  14999    NaN    0.0      0.0     0.0                    0.0   

       D_standard_error  D_max  D_min  K_sum  K_count  K_mean  \
0                   0.0   -1.0    0.0    0.0      0.0     0.0   
1                   0.0   -1.0    0.0    0.0      0.0     0.0   
2                   0.0   -1.0    0.0    0.0      0.0     0.0   
3                   0.0   -1.0    0.0    0.0      0.0     0.0   
4                   0.0   -1.0    0.0    0.0      0.0     0.0   
...                 ...    ...    ...    ...      ...     ...   
14995               0.0   -1.0    0.0    0.0      0.0     0.0   
14996               0.0   -1.0    0.0    0.0      0.0     0.0   
14997               0.0   -1.0    0.0    0.0      0.0     0.0   
14998               0.0   -1.0    0.0    0.0      0.0     0.0   
14999               0.0   -1.0    0.0    0.0      0.0     0.0   

       K_squarred_difference  K_standard_error  K_max  K_min  
0                        0.0               0.0   -1.0    0.0  
1                        0.0               0.0   -1.0    0.0  
2                        0.0               0.0   -1.0    0.0  
3                        0.0               0.0   -1.0    0.0  
4                        0.0               0.0   -1.0    0.0  
...                      ...               ...    ...    ...  
14995                    0.0               0.0   -1.0    0.0  
14996                    0.0               0.0   -1.0    0.0  
14997                    0.0               0.0   -1.0    0.0  
14998                    0.0               0.0   -1.0    0.0  
14999                    0.0               0.0   -1.0    0.0  

[15000 rows x 16 columns]

In [4]:
G = nx.from_pandas_edgelist(sim,source='id1',target='id2',edge_attr='sim')
class_dict = pd.Series(train['class'].values, index=train['id']).to_dict()
nx.set_node_attributes(G, class_dict, 'class')

In [5]:
pred_classes = pred_classes.set_index('id')
pred_classes

class  D_sum  D_count  D_mean  D_squarred_difference  D_standard_error  \
id                                                                              
0        NaN    0.0      0.0     0.0                    0.0               0.0   
1        NaN    0.0      0.0     0.0                    0.0               0.0   
2        NaN    0.0      0.0     0.0                    0.0               0.0   
3        NaN    0.0      0.0     0.0                    0.0               0.0   
4        NaN    0.0      0.0     0.0                    0.0               0.0   
...      ...    ...      ...     ...                    ...               ...   
14995    NaN    0.0      0.0     0.0                    0.0               0.0   
14996    NaN    0.0      0.0     0.0                    0.0               0.0   
14997    NaN    0.0      0.0     0.0                    0.0               0.0   
14998    NaN    0.0      0.0     0.0                    0.0               0.0   
14999    NaN    0.0      0.0     0.0                    0.0               0.0   

       D_max  D_min  K_sum  K_count  K_mean  K_squarred_difference  \
id                                                                   
0       -1.0    0.0    0.0      0.0     0.0                    0.0   
1       -1.0    0.0    0.0      0.0     0.0                    0.0   
2       -1.0    0.0    0.0      0.0     0.0                    0.0   
3       -1.0    0.0    0.0      0.0     0.0                    0.0   
4       -1.0    0.0    0.0      0.0     0.0                    0.0   
...      ...    ...    ...      ...     ...                    ...   
14995   -1.0    0.0    0.0      0.0     0.0                    0.0   
14996   -1.0    0.0    0.0      0.0     0.0                    0.0   
14997   -1.0    0.0    0.0      0.0     0.0                    0.0   
14998   -1.0    0.0    0.0      0.0     0.0                    0.0   
14999   -1.0    0.0    0.0      0.0     0.0                    0.0   

       K_standard_error  K_max  K_min  
id                                     
0                   0.0   -1.0    0.0  
1                   0.0   -1.0    0.0  
2                   0.0   -1.0    0.0  
3                   0.0   -1.0    0.0  
4                   0.0   -1.0    0.0  
...                 ...    ...    ...  
14995               0.0   -1.0    0.0  
14996               0.0   -1.0    0.0  
14997               0.0   -1.0    0.0  
14998               0.0   -1.0    0.0  
14999               0.0   -1.0    0.0  

[15000 rows x 15 columns]

In [6]:
# Function to perform BFS with custom similarity condition and store edge data
def custom_bfs(G, start_node, similarity_threshold=0.8, condition='>='):
    # Initialize the queue for BFS
    visited = set()
    queue = deque([start_node])
    visited.add(start_node)
    
    # List to store the results [old_vertice, new_vertice, similarity]
    edge_data = []
    
    while queue:
        node = queue.popleft()
        
        # Check neighbors
        for neighbor in G.neighbors(node):
            # Get the similarity score of the edge (node, neighbor)
            similarity = G[node][neighbor].get('sim', 0)
            
            # Check if the edge meets the similarity condition
            if (condition == '>=') and (similarity >= similarity_threshold):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
                    edge_data.append([node, neighbor, similarity])
                    #print(f"  Adding {neighbor} with similarity {similarity}")
            
            elif (condition == '<=') and (similarity <= similarity_threshold):
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append(neighbor)
                    edge_data.append([node, neighbor, similarity])
                    #print(f"  Adding {neighbor} with similarity {similarity}")
    
    # Return the list of edges with their similarity values
    return edge_data

# Function to perform BFS and track breadth (level) of each node
def bfs_with_breadth(G, start_node, similarity_threshold=0.8, condition='>='):
    # Initialize the queue for BFS
    queue = deque([(start_node, 0)])  # Each element in the queue is a tuple: (node, breadth)
    visited = set([start_node])      # Track visited nodes
    levels = {start_node: 0}         # Dictionary to store the level (breadth) of each node
    edge_data = []                   # To store [old_node, new_node, similarity, breadth]
    
    while queue:
        node, breadth = queue.popleft()
        # print(f"Visiting node: {node}, Breadth: {breadth}")
        
        # Check neighbors
        for neighbor in G.neighbors(node):
            # Get the similarity score of the edge
            similarity = G[node][neighbor].get('sim', 0)
            
            # Check if the edge meets the similarity condition
            if (condition == '>=') and (similarity >= similarity_threshold) or \
               (condition == '<=') and (similarity <= similarity_threshold):
                
                if neighbor not in visited:
                    visited.add(neighbor)
                    new_breadth = breadth + 1
                    levels[neighbor] = new_breadth
                    queue.append((neighbor, new_breadth))
                    edge_data.append([node, neighbor, similarity, new_breadth])
                    # print(f"  Adding {neighbor} with similarity {similarity}, Breadth: {new_breadth}")
    
    # Return the levels and the edge data
    return edge_data

def bfs_with_cumulative_similarity(G, start_node, similarity_threshold=0.8, condition='>='):
    # Initialize the queue for BFS
    queue = deque([(start_node, 0, 1.0)])  # Each element is a tuple: (node, breadth, cumulative_similarity)
    visited = set([start_node])            # Track visited nodes
    edge_data = []                         # To store [old_node, new_node, edge_similarity, cumulative_similarity, breadth]
    
    while queue:
        node, breadth, cumulative_similarity = queue.popleft()
        
        # Check neighbors
        for neighbor in G.neighbors(node):
            # Get the similarity score of the edge
            edge_similarity = G[node][neighbor].get('sim', 0)
            
            # Calculate new cumulative similarity
            new_cumulative_similarity = cumulative_similarity * edge_similarity
            
            # Check if the edge meets the similarity condition
            if (condition == '>=') and (new_cumulative_similarity >= similarity_threshold) or \
               (condition == '<=') and (new_cumulative_similarity <= similarity_threshold):
                
                if neighbor not in visited:
                    visited.add(neighbor)
                    new_breadth = breadth + 1
                    queue.append((neighbor, new_breadth, new_cumulative_similarity))
                    edge_data.append([node, neighbor, new_cumulative_similarity, new_breadth])
    
    # Return the edge data
    return edge_data

In [7]:
i = 0
# Вершины, которые нужно исключить

# Создаем подграф, исключая определенные вершины

Make_a_class = pd.DataFrame(bfs_with_cumulative_similarity(G, i, similarity_threshold=0, condition='<='), columns=['id1', 'id2', 'sim', 'breadth'])
simularity = Make_a_class[['id2', 'sim', 'breadth']].rename({'id2': 'id'}, axis=1).set_index('id')

if train[train.id == i]['class'].values[0] == 'D':
    pred_classes.loc[simularity.index, 'D_sum'] += simularity.sim
    pred_classes.loc[simularity.index, 'D_count'] += 1 / simularity.breadth
elif train[train.id == i]['class'].values[0] == 'K':
    pred_classes.loc[simularity.index, 'K_sum'] += simularity.sim
    pred_classes.loc[simularity.index, 'K_count'] += 1 / simularity.breadth

print(i)
for i in range(1, 10000):

    Make_a_class = pd.DataFrame(bfs_with_cumulative_similarity(G, i, similarity_threshold=0, condition='<='), columns=['id1', 'id2', 'sim', 'breadth'])
    simularity = Make_a_class[['id2', 'sim', 'breadth']].rename({'id2': 'id'}, axis=1).set_index('id')

    if train[train.id == i]['class'].values[0] == 'D':
        pred_classes.loc[simularity.index, 'D_sum'] += simularity.sim 
        pred_classes.loc[simularity.index, 'D_count'] += 1 / simularity.breadth # (сделано) Потенциально для расчёта среднего лучше ещё делить на ширину типо, элемент не сильно далёкий на похожесть влияет сильнее чем далёкий.
        pred_classes.loc[simularity.index, 'D_squarred_difference'] += (simularity.sim - first_data.loc[simularity.index, 'K_mean'])**2
        pred_classes.loc[simularity.index, 'D_max'] = np.maximum(simularity.sim, pred_classes.loc[simularity.index, 'D_max'])
        pred_classes.loc[simularity.index, 'D_min'] = np.minimum(simularity.sim, pred_classes.loc[simularity.index, 'D_min'])
    elif train[train.id == i]['class'].values[0] == 'K':
        pred_classes.loc[simularity.index, 'K_sum'] += simularity.sim
        pred_classes.loc[simularity.index, 'K_count'] += 1 / simularity.breadth
        pred_classes.loc[simularity.index, 'K_squarred_difference'] += (simularity.sim - first_data.loc[simularity.index, 'K_mean'])**2
        pred_classes.loc[simularity.index, 'K_max'] = np.maximum(simularity.sim, pred_classes.loc[simularity.index, 'K_max'])
        pred_classes.loc[simularity.index, 'K_min'] = np.minimum(simularity.sim, pred_classes.loc[simularity.index, 'K_min'])
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
pred_classes.to_csv('advanced_minus_data_for_ml.csv')

In [8]:
pred_classes.loc[simularity.index].sort_index()

class     D_sum   D_count  D_mean     K_sum   K_count  K_mean
id                                                                  
0        NaN -0.442547  4.166667     0.0 -1.321171  3.833333     0.0
1        NaN -0.971499  5.000000     0.0 -0.488439  3.000000     0.0
2        NaN -0.336179  4.166667     0.0 -0.860829  3.666667     0.0
3        NaN -0.762742  4.833333     0.0 -0.366936  3.333333     0.0
4        NaN -1.491149  4.500000     0.0 -1.373211  4.166667     0.0
...      ...       ...       ...     ...       ...       ...     ...
14995    NaN -0.334940  4.666667     0.0 -1.222210  4.000000     0.0
14996    NaN -1.412877  5.666667     0.0 -0.158426  3.500000     0.0
14997    NaN -0.666291  5.166667     0.0 -0.439999  4.166667     0.0
14998    NaN -1.158809  5.000000     0.0 -0.568782  4.333333     0.0
14999    NaN -2.065845  5.000000     0.0 -0.135231  3.166667     0.0

[14999 rows x 7 columns]

In [9]:
pred_classes = pd.read_csv('first_minus_data_for_ml.csv', index_col=0)
pred_classes

class       D_sum      D_count  D_mean       K_sum      K_count  K_mean
id                                                                            
0        NaN -405.776287  2756.500000     0.0 -367.086575  1339.000000     0.0
1        NaN -665.306470  2913.833333     0.0 -164.958415  1236.166667     0.0
2        NaN -349.802807  2779.833333     0.0 -226.382758  1319.666667     0.0
3        NaN -451.004127  2890.333333     0.0 -135.502683  1256.000000     0.0
4        NaN -445.659327  2794.000000     0.0 -381.058482  1352.333333     0.0
...      ...         ...          ...     ...         ...          ...     ...
14995    NaN -436.845293  2759.333333     0.0 -412.016401  1344.333333     0.0
14996    NaN -552.881760  2931.500000     0.0 -137.054831  1231.166667     0.0
14997    NaN -487.924329  2918.500000     0.0 -171.393176  1261.166667     0.0
14998    NaN -455.885051  2907.833333     0.0 -214.564370  1316.666667     0.0
14999    NaN -907.451780  2908.833333     0.0 -206.534707  1193.666667     0.0

[15000 rows x 7 columns]

In [14]:
pred_classes['D_mean'] = pred_classes.apply(lambda x: x.D_sum / x.D_count, axis=1) * 100 # Percentage
pred_classes['K_mean'] = pred_classes.apply(lambda x: x.K_sum / x.K_count, axis=1) * 100 # Percentage

In [15]:
pred_classes.max()

class              NaN
D_sum       -40.298445
D_count    3008.500000
D_mean       -1.407314
K_sum        -9.405738
K_count    1392.333333
K_mean       -0.788962
dtype: float64

In [16]:
train = train.set_index('id')
train

class
id        
0        D
1        K
2        D
3        K
4        D
...    ...
9995     D
9996     K
9997     K
9998     D
9999     K

[10000 rows x 1 columns]

In [17]:
pred_classes.loc[train.index, 'class'] = train['class']
pred_classes

/tmp/ipykernel_57567/155788887.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['D' 'K' 'D' ... 'K' 'D' 'K']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  pred_classes.loc[train.index, 'class'] = train['class']


class       D_sum      D_count     D_mean       K_sum      K_count  \
id                                                                         
0         D -405.776287  2756.500000 -14.720707 -367.086575  1339.000000   
1         K -665.306470  2913.833333 -22.832688 -164.958415  1236.166667   
2         D -349.802807  2779.833333 -12.583589 -226.382758  1319.666667   
3         K -451.004127  2890.333333 -15.603879 -135.502683  1256.000000   
4         D -445.659327  2794.000000 -15.950584 -381.058482  1352.333333   
...     ...         ...          ...        ...         ...          ...   
14995   NaN -436.845293  2759.333333 -15.831552 -412.016401  1344.333333   
14996   NaN -552.881760  2931.500000 -18.860029 -137.054831  1231.166667   
14997   NaN -487.924329  2918.500000 -16.718325 -171.393176  1261.166667   
14998   NaN -455.885051  2907.833333 -15.677826 -214.564370  1316.666667   
14999   NaN -907.451780  2908.833333 -31.196417 -206.534707  1193.666667   

          K_mean  
id                
0     -27.414979  
1     -13.344351  
2     -17.154541  
3     -10.788430  
4     -28.177852  
...          ...  
14995 -30.648381  
14996 -11.132110  
14997 -13.590050  
14998 -16.296028  
14999 -17.302545  

[15000 rows x 7 columns]

# XGBoost!!!!!!


In [18]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import optuna
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score

In [19]:
X, y = pred_classes.loc[0:9999].drop(columns=['class']), pred_classes.loc[0:9999]['class'].replace({'D': 0, 'K': 1})
X_test = pred_classes.loc[10000:14999]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

/tmp/ipykernel_57567/3718468339.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X, y = pred_classes.loc[0:9999].drop(columns=['class']), pred_classes.loc[0:9999]['class'].replace({'D': 0, 'K': 1})


In [20]:
X_train

D_sum      D_count     D_mean       K_sum      K_count     K_mean
id                                                                          
9254 -330.743862  2802.166667 -11.803147 -242.939910  1324.166667 -18.346626
1561 -409.713383  2755.166667 -14.870730 -365.057229  1351.000000 -27.021261
1670 -536.507237  2863.166667 -18.738247 -190.142901  1262.333333 -15.062812
6087 -755.664884  2901.333333 -26.045435 -170.473620  1196.000000 -14.253647
6669 -788.317125  2936.000000 -26.850038 -172.858438  1218.500000 -14.186166
...          ...          ...        ...         ...          ...        ...
5734 -346.887208  2762.666667 -12.556245 -267.561059  1328.500000 -20.140087
5191 -689.164188  2906.500000 -23.711137 -166.831534  1208.666667 -13.802940
5390 -533.332123  2966.500000 -17.978497 -172.882713  1273.000000 -13.580732
860  -654.368796  2899.000000 -22.572225 -155.804649  1219.500000 -12.776109
7270 -450.667660  2891.500000 -15.585947 -153.758249  1254.500000 -12.256536

[8000 rows x 6 columns]

In [21]:
X_val

D_sum      D_count     D_mean       K_sum      K_count     K_mean
id                                                                          
6252 -283.945932  2813.333333 -10.092865 -134.287022  1298.833333 -10.339050
4684 -270.585802  2812.000000  -9.622539 -185.043162  1324.166667 -13.974311
1731 -390.050896  2888.666667 -13.502800 -171.205335  1295.666667 -13.213687
4742 -534.566082  2929.166667 -18.249767 -216.010123  1299.333333 -16.624689
4521 -242.727823  2739.666667  -8.859758 -189.095695  1323.666667 -14.285749
...          ...          ...        ...         ...          ...        ...
6412 -404.528736  2683.666667 -15.073733 -307.718018  1287.000000 -23.909714
8285 -334.532486  2752.166667 -12.155241 -273.388617  1327.166667 -20.599419
7853 -340.162737  2749.666667 -12.371054 -309.780336  1340.166667 -23.115060
1095 -403.011450  2846.500000 -14.158140 -326.333166  1365.333333 -23.901355
6929 -650.138542  2815.333333 -23.092773 -158.797984  1186.833333 -13.379973

[2000 rows x 6 columns]

In [22]:
y_train

id
9254    1
1561    0
1670    0
6087    1
6669    1
       ..
5734    0
5191    1
5390    1
860     1
7270    0
Name: class, Length: 8000, dtype: int64

In [23]:
y_val

id
6252    0
4684    0
1731    0
4742    1
4521    1
       ..
6412    0
8285    0
7853    0
1095    0
6929    1
Name: class, Length: 2000, dtype: int64

In [24]:
clf = XGBClassifier(n_estimators = 500, learning_rate = 0.01, max_depth = 6, early_stopping_rounds = 10, eval_metric = 'logloss', n_jobs=-1)

clf.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=True,
)

[0]	validation_0-logloss:0.61238	validation_1-logloss:0.62776
[1]	validation_0-logloss:0.60969	validation_1-logloss:0.62511
[2]	validation_0-logloss:0.60705	validation_1-logloss:0.62251
[3]	validation_0-logloss:0.60445	validation_1-logloss:0.61998
[4]	validation_0-logloss:0.60191	validation_1-logloss:0.61752
[5]	validation_0-logloss:0.59943	validation_1-logloss:0.61510
[6]	validation_0-logloss:0.59699	validation_1-logloss:0.61277
[7]	validation_0-logloss:0.59462	validation_1-logloss:0.61047
[8]	validation_0-logloss:0.59229	validation_1-logloss:0.60824
[9]	validation_0-logloss:0.59000	validation_1-logloss:0.60606
[10]	validation_0-logloss:0.58777	validation_1-logloss:0.60391
[11]	validation_0-logloss:0.58557	validation_1-logloss:0.60180
[12]	validation_0-logloss:0.58342	validation_1-logloss:0.59977
[13]	validation_0-logloss:0.58131	validation_1-logloss:0.59779
[14]	validation_0-logloss:0.57925	validation_1-logloss:0.59576
[15]	validation_0-logloss:0.57722	validation_1-logloss:0.59385
[1

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=-1, num_parallel_tree=None, random_state=None, ...)

In [25]:
f1_score(y_val, clf.predict(X_val))

np.float64(0.5778611632270169)

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, clf.predict(X_val))

0.775

In [29]:
X_test.loc[:, 'class'] = clf.predict(X_test.drop(columns=['class']))
X_test

class       D_sum      D_count     D_mean       K_sum      K_count  \
id                                                                         
10000     0 -385.027232  2726.333333 -14.122530 -380.179475  1340.500000   
10001     1 -779.640943  2891.166667 -26.966309 -164.353876  1189.333333   
10002     0 -369.060369  2806.166667 -13.151762 -319.313641  1347.000000   
10003     0 -377.983456  2823.000000 -13.389425 -271.071463  1326.666667   
10004     0 -406.641055  2771.500000 -14.672237 -340.121594  1328.166667   
...     ...         ...          ...        ...         ...          ...   
14995     0 -436.845293  2759.333333 -15.831552 -412.016401  1344.333333   
14996     1 -552.881760  2931.500000 -18.860029 -137.054831  1231.166667   
14997     0 -487.924329  2918.500000 -16.718325 -171.393176  1261.166667   
14998     0 -455.885051  2907.833333 -15.677826 -214.564370  1316.666667   
14999     1 -907.451780  2908.833333 -31.196417 -206.534707  1193.666667   

          K_mean  
id                
10000 -28.361020  
10001 -13.818992  
10002 -23.705541  
10003 -20.432522  
10004 -25.608352  
...          ...  
14995 -30.648381  
14996 -11.132110  
14997 -13.590050  
14998 -16.296028  
14999 -17.302545  

[5000 rows x 7 columns]

In [30]:
predict = X_test.replace({0: 'D', 1: 'K'}).reset_index()[['id', 'class']]

In [32]:
predict.to_csv('minus_ml_predict2.csv', index=False)

In [73]:
predict['class'].value_counts()

class
D    4050
K     950
Name: count, dtype: int64

## Predict_proba


In [33]:
predo = pd.read_csv('ml_predict1.csv')
predo.loc[:, 'class'] = clf.predict_proba(X_test.drop(columns=['class'])).T[1]

In [38]:
predo['class'].apply(lambda x: x > 0.43).value_counts()

class
False    3862
True     1138
Name: count, dtype: int64

In [39]:
predo['class'] = predo['class'].apply(lambda x: x > 0.43)

In [40]:
predo['class'].value_counts()

class
False    3862
True     1138
Name: count, dtype: int64

In [43]:
predo.replace({False: "D", True: "K"}).to_csv("minus_Ml_predict2.csv", index=False)

In [42]:
predo

id  class
0     10000  False
1     10001   True
2     10002  False
3     10003  False
4     10004  False
...     ...    ...
4995  14995  False
4996  14996   True
4997  14997  False
4998  14998  False
4999  14999   True

[5000 rows x 2 columns]

# Old Post-Processing


In [117]:
pred_classes.loc[:, 'class'] = 'D'
pred_classes.loc[pred_classes[(pred_classes.K_mean > pred_classes.D_mean) & (pred_classes.K_mean > 0.06)].index, 'class'] = 'K'
# pred_classes.loc[pred_classes[pred_classes.K_mean <= pred_classes.D_mean].index, 'class'] = 'D'

In [94]:
pred_classes.loc[pred_classes[pred_classes.K_mean > 0.063].index, 'class'] = 'K'
pred_classes.loc[pred_classes[pred_classes.K_mean <= 0.063].index, 'class'] = 'D'

In [124]:
pred_classes['class'].value_counts()

class
D    3846
K    1154
Name: count, dtype: int64

In [119]:
pred_classes

class       D_sum  D_count    D_mean       K_sum  K_count    K_mean
id                                                                       
10000     D  599.547474   6912.0  0.086740  137.069201   3088.0  0.044388
10001     K  284.393159   6912.0  0.041145  252.153682   3088.0  0.081656
10002     D  550.443102   6912.0  0.079636  131.340401   3088.0  0.042533
10003     D  479.328720   6912.0  0.069347  134.952938   3088.0  0.043702
10004     D  593.770419   6912.0  0.085904  144.352726   3088.0  0.046746
...     ...         ...      ...       ...         ...      ...       ...
14995     D  708.458124   6912.0  0.102497  159.778180   3088.0  0.051742
14996     D  173.067118   6912.0  0.025039  165.892871   3088.0  0.053722
14997     D  279.725701   6912.0  0.040470  161.493077   3088.0  0.052297
14998     D  338.116477   6912.0  0.048917  169.085405   3088.0  0.054756
14999     K  386.541760   6912.0  0.055923  381.013445   3088.0  0.123385

[5000 rows x 7 columns]

In [19]:
pred_classes[pred_classes['class'].isna()]

class       D_sum      D_count    D_mean       K_sum      K_count  \
id                                                                         
0        NaN  777.614282  2975.333333  0.261354  189.940639  1218.666667   
1        NaN  379.111613  2749.833333  0.137867  320.586678  1293.166667   
2        NaN  464.316126  2942.166667  0.157814  156.871831  1235.500000   
3        NaN  294.214732  2791.833333  0.105384  211.299259  1292.666667   
4        NaN  820.776927  3009.000000  0.272774  196.406639  1249.500000   
...      ...         ...          ...       ...         ...          ...   
14995    NaN  892.810953  3019.000000  0.295731  194.093758  1227.333333   
14996    NaN  307.139350  2754.333333  0.111511  273.660614  1323.500000   
14997    NaN  367.227889  2820.833333  0.130184  219.240044  1303.333333   
14998    NaN  471.464040  2966.333333  0.158938  215.806324  1298.000000   
14999    NaN  460.232824  2657.666667  0.173172  460.621661  1303.666667   

         K_mean  
id               
0      0.155859  
1      0.247908  
2      0.126970  
3      0.163460  
4      0.157188  
...         ...  
14995  0.158143  
14996  0.206770  
14997  0.168215  
14998  0.166261  
14999  0.353328  

[15000 rows x 7 columns]

In [121]:
pred_classes[pred_classes['class'] == 0]

Empty DataFrame
Columns: [class, D_sum, D_count, D_mean, K_sum, K_count, K_mean]
Index: []

In [123]:
pred_classes.reset_index()[['id', 'class']].to_csv('bfs_third_version.csv', index=False)

In [12]:
pred_classes.loc[simularity.index].sort_index()

class     D_sum  D_count  D_mean     K_sum  K_count  K_mean
id                                                                
10037    NaN  0.721931      1.0     0.0  3.601700      5.0     0.0
10067    NaN  1.404952      2.0     0.0  1.978038      3.0     0.0
10160    NaN  2.672333      4.0     0.0  1.475610      2.0     0.0
10198    NaN  3.382734      5.0     0.0  0.000000      0.0     0.0
10440    NaN  3.489978      5.0     0.0  0.000000      0.0     0.0
...      ...       ...      ...     ...       ...      ...     ...
14655    NaN  3.739731      5.0     0.0  0.000000      0.0     0.0
14660    NaN  2.958026      4.0     0.0  0.608879      1.0     0.0
14736    NaN  3.641358      5.0     0.0  0.000000      0.0     0.0
14787    NaN  1.468872      2.0     0.0  0.716277      1.0     0.0
14801    NaN  4.909661      6.0     0.0  0.000000      0.0     0.0

[100 rows x 7 columns]

In [13]:
pred_classes[(pred_classes.D_count > 0) | (pred_classes.K_count > 0)]

class     D_sum  D_count  D_mean     K_sum  K_count  K_mean
id                                                                
10000    NaN  2.395088      3.0     0.0  0.000000      0.0     0.0
10001    NaN  0.000000      0.0     0.0  1.625563      2.0     0.0
10002    NaN  3.488712      5.0     0.0  0.000000      0.0     0.0
10003    NaN  1.230105      2.0     0.0  0.000000      0.0     0.0
10004    NaN  4.015353      6.0     0.0  0.000000      0.0     0.0
...      ...       ...      ...     ...       ...      ...     ...
14989    NaN  0.000000      0.0     0.0  2.077058      3.0     0.0
14990    NaN  1.986381      3.0     0.0  0.000000      0.0     0.0
14995    NaN  5.370993      8.0     0.0  0.000000      0.0     0.0
14998    NaN  1.234757      2.0     0.0  0.000000      0.0     0.0
14999    NaN  0.767606      1.0     0.0  3.901501      5.0     0.0

[3540 rows x 7 columns]

In [14]:
pred_classes

class     D_sum  D_count  D_mean     K_sum  K_count  K_mean
id                                                                
10000    NaN  2.395088      3.0     0.0  0.000000      0.0     0.0
10001    NaN  0.000000      0.0     0.0  1.625563      2.0     0.0
10002    NaN  3.488712      5.0     0.0  0.000000      0.0     0.0
10003    NaN  1.230105      2.0     0.0  0.000000      0.0     0.0
10004    NaN  4.015353      6.0     0.0  0.000000      0.0     0.0
...      ...       ...      ...     ...       ...      ...     ...
14995    NaN  5.370993      8.0     0.0  0.000000      0.0     0.0
14996    NaN  0.000000      0.0     0.0  0.000000      0.0     0.0
14997    NaN  0.000000      0.0     0.0  0.000000      0.0     0.0
14998    NaN  1.234757      2.0     0.0  0.000000      0.0     0.0
14999    NaN  0.767606      1.0     0.0  3.901501      5.0     0.0

[5000 rows x 7 columns]

In [16]:
pred_classes['class'].isna().sum()

np.int64(5000)

In [39]:
pred_classes[pred_classes.index.isin(simularity.index)]

class     D_sum  D_count  D_mean  K_sum  K_count  K_mean
id                                                             
10000    NaN  0.854674      0.0     0.0    0.0      0.0     0.0
10002    NaN  0.725023      0.0     0.0    0.0      0.0     0.0
10004    NaN  0.601816      0.0     0.0    0.0      0.0     0.0
10022    NaN  0.832458      0.0     0.0    0.0      0.0     0.0
10025    NaN  0.628172      0.0     0.0    0.0      0.0     0.0
...      ...       ...      ...     ...    ...      ...     ...
14912    NaN  0.713893      0.0     0.0    0.0      0.0     0.0
14939    NaN  0.878399      0.0     0.0    0.0      0.0     0.0
14947    NaN  0.802698      0.0     0.0    0.0      0.0     0.0
14950    NaN  0.611281      0.0     0.0    0.0      0.0     0.0
14965    NaN  0.669374      0.0     0.0    0.0      0.0     0.0

[438 rows x 7 columns]

In [71]:
pred_classes['class'].value_counts()

class
K    2611
D    2115
Name: count, dtype: int64

In [84]:
pred_classes[pred_classes['class'].isna()]

id class
34    10034   NaN
44    10044   NaN
80    10080   NaN
86    10086   NaN
125   10125   NaN
...     ...   ...
4972  14972   NaN
4974  14974   NaN
4978  14978   NaN
4991  14991   NaN
4994  14994   NaN

[274 rows x 2 columns]

In [73]:
pred_classes.isna()['class'].sum()

np.int64(274)

In [ ]:
pred_classes.to_csv('bfs_second_version.csv', index=False)

In [34]:
na = pred_classes[pred_classes.isna()['class']].id.values
na

array([10003, 10005, 10009, ..., 14994, 14996, 14997])